In [1]:
import cv2


COLORS = {'blue': (255, 0, 0), 'green': (0, 255, 0), 'red': (0, 0, 255), 'yellow':(0, 255, 255),
'magenta': (255, 0, 255), 'cyan': (255, 255, 0), 'white': (255, 255,255),'orange':(0,165,255),
'amber': (255, 191, 0), 'gray': (125, 125, 125), 'dark_gray': (50, 50, 50),
'light_gray': (220, 220,220), 'black': (0,0,0)}


class Draw():
    
    def __init__(self,radius=5,color='red'):
        self.points = []
        self.radius = radius
        self.color = color
        
    def draw(self):
        for points in self.points:
            cv2.circle(img,(points[0],points[1]),self.radius,COLORS[points[2]],-1)
            
    def add_points(self,cx,cy,color):
        self.points.append([cx,cy,color])
        
    def delete_points(self,cx,cy):
        points_copy = [points for points in self.points if (points[0]!=cx) and (points[1]!=cy)]
        self.points = points_copy
        
    def clear(self):
        self.points = []
        

In [2]:
import cv2
import mediapipe as mp
import numpy as np

class HandDetector():
    
    def __init__(self):
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(max_num_hands=1)
        self.mpDraw = mp.solutions.drawing_utils     # For Drawing in image
        
    def findHands(self,img,draw=True):
        imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)    # Convert BGR to RGB
        self.results = self.hands.process(imgRGB)      # Landmarks
        
        if self.results.multi_hand_landmarks:  
            for handLms in self.results.multi_hand_landmarks:   
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS) # To Draw all connections
        return img
    
    def findPosition(self,img):
        lmList = []
        
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                for ids,lms in enumerate(handLms.landmark):
                    h,w,c = img.shape                   # Dimension of image
                    cx, cy = int(lms.x * w), int(lms.y * h)     # centre of each points
                    lmList.append([ids, cx, cy])     # Landmarks
        
        return lmList

In [18]:
import cv2
colors = ['yellow','magenta','orange','cyan']

cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,1080)

draw = False
detector = HandDetector()

paint = Draw(radius=10)
while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,None,fx=1.8,fy=1.5)
    img = detector.findHands(frame)
    lmlist = detector.findPosition(img)
    
    
    
    cv2.rectangle(img,(10,20),(150,100),(255,255,255),-1)
    cv2.rectangle(img,(160,20),(310,100),(255,255,255),-1)
    cv2.rectangle(img,(320,20),(470,100),(255,255,255),-1)
    cv2.rectangle(img,(480,20),(630,100),(255,255,255),-1)
    cv2.rectangle(img,(640,20),(790,100),(255,255,255),-1)
    cv2.rectangle(img,(800,20),(950,100),(255,255,255),-1)
    cv2.rectangle(img,(960,20),(1110,100),(255,255,255),-1)
    
    
    cv2.putText(img,'CLEAR',(50,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,0,0))
    cv2.putText(img,'BLUE',(200,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(255,0,0))
    cv2.putText(img,'GREEN',(370,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,255,0))
    cv2.putText(img,'RED',(530,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,0,255))
    cv2.putText(img,'RANDOM',(680,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,165,255))
    cv2.putText(img,'DRAW',(850,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,0,0))
    cv2.putText(img,'ERASER',(1000,65),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color=(0,0,0))
    
    
    if len(lmlist)!=0:
        if (lmlist[8][1] <= 150 and lmlist[8][1]>=10) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
            paint.clear()
        elif (lmlist[8][1] <= 310 and lmlist[8][1]>=160) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
#             paint.add_points(lmlist[8][1],lmlist[8][2],'blue')
            if draw:
                paint.color='blue'
        elif (lmlist[8][1] <= 470 and lmlist[8][1]>=320) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
#             paint.add_points(lmlist[8][1],lmlist[8][2],'green')
            if draw:
                paint.color='green'
        elif (lmlist[8][1] <= 630 and lmlist[8][1]>=480) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
#             paint.add_points(lmlist[8][1],lmlist[8][2],'red')
            if draw:
                paint.color = 'red'
        elif (lmlist[8][1] <= 790 and lmlist[8][1]>=640) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
            if draw:
                paint.color = np.random.choice(colors)
            
        elif (lmlist[8][1] <= 950 and lmlist[8][1]>=800) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
            draw = True
            
        elif (lmlist[8][1] <= 1110 and lmlist[8][1]>=960) and (lmlist[8][2]<=100 and lmlist[8][2]>=20):
            draw = False
        
        elif draw==False:
            paint.delete_points(lmlist[8][1],lmlist[8][2])
    
        else:
            if draw:
                paint.add_points(lmlist[8][1],lmlist[8][2],paint.color)
    paint.draw()
    cv2.imshow('Virtual Paint',img)
    if cv2.waitKey(2)==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()